In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Veriyi oku 
df = pd.read_csv("C:\\Users\\yunus\\Desktop\\Proje Fikirleri\\anomali-tespiti\\data\\raw\\cicids2017_sample_cleaned.csv")


In [2]:
# Etiketleri sayıya çevir
le = LabelEncoder()
df['Label_encoded'] = le.fit_transform(df['Label'])

# Etiket dönüşümünü kontrol et
print("Etiket Dönüşümleri:")
print(df[['Label', 'Label_encoded']].drop_duplicates())


Etiket Dönüşümleri:
         Label  Label_encoded
0       BENIGN              0
6     PortScan              3
12        DDoS              2
49  BruteForce              1


In [3]:
# Normalize edilecek özellik sütunları
features = [
    'Flow Duration',
    'Total Fwd Packets',
    'Total Backward Packets',
    'Total Length of Fwd Packets',
    'Total Length of Bwd Packets'
]

# StandardScaler ile normalize et
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

# DataFrame'e çevir ve etiket sütununu ekle
X = pd.DataFrame(X_scaled, columns=features)
y = df['Label_encoded']


In [4]:
# Eğitim ve test verilerini ayır
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Eğitim veri seti boyutu:", X_train.shape)
print("Test veri seti boyutu:", X_test.shape)


Eğitim veri seti boyutu: (800, 5)
Test veri seti boyutu: (200, 5)


In [5]:
# Modeli tanımla ve eğit
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("Model eğitimi tamamlandı.")


Model eğitimi tamamlandı.


In [6]:
# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# Sınıflandırma raporu
print("🔍 Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Confusion matrix
print("📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


🔍 Sınıflandırma Raporu:
              precision    recall  f1-score   support

      BENIGN       0.73      0.99      0.84       145
  BruteForce       0.00      0.00      0.00         9
        DDoS       0.25      0.03      0.06        29
    PortScan       0.00      0.00      0.00        17

    accuracy                           0.72       200
   macro avg       0.24      0.26      0.22       200
weighted avg       0.57      0.72      0.62       200

📊 Confusion Matrix:
[[143   0   2   0]
 [  9   0   0   0]
 [ 28   0   1   0]
 [ 16   0   1   0]]


C:\Users\yunus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yunus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yunus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
# Dengesiz sınıfları dikkate alarak model oluştur
model_balanced = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model_balanced.fit(X_train, y_train)

# Yeni tahminler
y_pred_balanced = model_balanced.predict(X_test)

# Performansı değerlendir
print("🔁 Dengelenmiş Modelin Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_balanced, target_names=le.classes_))

print("📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_balanced))


🔁 Dengelenmiş Modelin Sınıflandırma Raporu:
              precision    recall  f1-score   support

      BENIGN       0.72      0.99      0.83       145
  BruteForce       0.00      0.00      0.00         9
        DDoS       0.00      0.00      0.00        29
    PortScan       0.00      0.00      0.00        17

    accuracy                           0.71       200
   macro avg       0.18      0.25      0.21       200
weighted avg       0.52      0.71      0.60       200

📊 Confusion Matrix:
[[143   0   1   1]
 [  9   0   0   0]
 [ 29   0   0   0]
 [ 17   0   0   0]]


C:\Users\yunus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yunus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yunus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
from imblearn.over_sampling import SMOTE

# SMOTE uygulayarak sınıfları dengele
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Yeni dengelenmiş veri ile eğitim/test seti oluştur
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

print("Dengelenmiş veri seti sınıf dağılımı:")
print(pd.Series(y_resampled).value_counts())


Dengelenmiş veri seti sınıf dağılımı:
Label_encoded
0    723
3    723
2    723
1    723
Name: count, dtype: int64


In [9]:
# Yeni model oluştur ve eğit
model_smote = RandomForestClassifier(n_estimators=100, random_state=42)
model_smote.fit(X_train, y_train)

# Tahmin yap
y_pred_smote = model_smote.predict(X_test)

# Performansı değerlendir
print("🎯 SMOTE Uygulandıktan Sonraki Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_smote, target_names=le.classes_))

print("📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_smote))


🎯 SMOTE Uygulandıktan Sonraki Sınıflandırma Raporu:
              precision    recall  f1-score   support

      BENIGN       0.76      0.61      0.68       145
  BruteForce       0.86      0.96      0.91       144
        DDoS       0.80      0.81      0.81       145
    PortScan       0.81      0.87      0.84       145

    accuracy                           0.81       579
   macro avg       0.81      0.81      0.81       579
weighted avg       0.81      0.81      0.81       579

📊 Confusion Matrix:
[[ 89   9  22  25]
 [  1 138   3   2]
 [ 20   5 118   2]
 [  7   8   4 126]]
